#CHARTS I HAVE EMBEDDE CHARTS BELOW

In [ ]:
!pip install pandas==2.0.3 plotly==5.15.0 openpyxl==3.1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.1.5
    Uninstalling openpyxl-3.1.5:
      Successfully uninstalled openpyxl-3.1.5
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.0.3 which is incompatible.
mizani 0.13.1 requires pand

#1. Revenue Distribution Analysis (Pareto Chart)

In [ ]:
import pandas as pd
import plotly.graph_objects as go

In [ ]:
# Replace 'your_file.xlsx' with the actual filename
df = pd.read_excel('KAREEMA MEDICAL STORE .xlsx')

In [ ]:
# Calculate revenue by category
cat_revenue = df.groupby('Therapeutic Tag')['Revenue'].sum().sort_values(ascending=False)
total_revenue = cat_revenue.sum()
cat_revenue_pct = cat_revenue / total_revenue * 100
cum_pct = cat_revenue_pct.cumsum()

# Create Pareto Chart
fig = go.Figure()

# Add bars
fig.add_trace(go.Bar(
    x=cat_revenue.index,
    y=cat_revenue.values,
    name='Revenue',
    marker_color='royalblue'
))

# Add line
fig.add_trace(go.Scatter(
    x=cat_revenue.index,
    y=cum_pct,
    mode='lines+markers',
    name='Cumulative %',
    marker=dict(size=8, color='red'),
    yaxis='y2'
))

# Layout
fig.update_layout(
    title='Pareto Chart: Revenue Contribution by Therapeutic Category',
    xaxis=dict(title='Therapeutic Category', tickangle=45),
    yaxis=dict(title='Revenue (₹)'),
    yaxis2=dict(
        title='Cumulative Percentage',
        titlefont=dict(color='red'),
        tickfont=dict(color='red'),
        overlaying='y',
        side='right',
        range=[0, 100]
    ),
    legend=dict(x=0.01, y=0.99),
    plot_bgcolor='white'
)

fig.show()


#3. Stockout Risk Assessment (Heat Map)

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Assuming you have already loaded your data into 'df' from 'KAREEMA MEDICAL STORE .xlsx'

# ... (previous code for Pareto Chart and Box Plot) ...

# Get the data (Replace with your actual data loading method)
# df = pd.read_excel('KAREEMA MEDICAL STORE .xlsx')  # If you haven't already loaded the data
df = df  # Keep the existing DataFrame 'df'

# Get top categories
top_categories = ['Antibiotic', 'NSAID', 'Antacid', 'Vitamin', 'Bronchodilator',
                  'Probiotic', 'Antihistamine', 'Analgesic']

# Create stockout risk measure
risk_df = df[df['Therapeutic Tag'].isin(top_categories)].copy()

# Calculate stockout risk based on closing quantity vs average issue quantity
risk_df['Stockout Risk'] = np.where(
    risk_df['CLOSING_QTY'] < 0,
    3,  # High risk if negative
    np.where(
        risk_df['CLOSING_QTY'] < risk_df['ISSUE_QTY'] * 0.1,
        2,  # Medium-high risk if less than 10% of issues
        np.where(
            risk_df['CLOSING_QTY'] < risk_df['ISSUE_QTY'] * 0.3,
            1,  # Medium risk if less than 30% of issues
            0   # Low risk
        )
    )
)

# Aggregate risk by category
risk_agg = risk_df.groupby('Therapeutic Tag')['Stockout Risk'].value_counts().unstack().fillna(0)
risk_agg = risk_agg.div(risk_agg.sum(axis=1), axis=0) * 100  # Convert to percentage

# Create heat map
fig = go.Figure(data=go.Heatmap(
    z=risk_agg.values,
    x=['Low Risk', 'Medium Risk', 'Medium-High Risk', 'High Risk'],
    y=risk_agg.index,
    colorscale='YlOrRd',
    text=np.round(risk_agg.values, 1),
    texttemplate="%{text}%",
    textfont={"size":10}
))

fig.update_layout(
    title='Stockout Risk Profile by Therapeutic Category',
    xaxis_title='Stockout Risk Level',
    yaxis_title='Therapeutic Category',
    plot_bgcolor='white'
)

fig.show()

#3. Weekly Sales Trend Analysis (September 2024 - February 2025)source


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta

import plotly.express as px

# ... (your previous code) ...

# Create scatter plot
fig = px.scatter(df,
                x='Revenue',
                y='DUMP_QTY',
                color='Therapeutic Tag',
                # Ensure 'size' values are non-negative
                size=df['ISSUE_QTY'].clip(lower=0),  # Clip negative values to 0
                hover_data=['SKU', 'CLOSING_QTY'],
                title='Revenue vs Dump Quantity by Category')

# ... (rest of your code) ...

# Create weekly dates for entire period
start_date = datetime(2024, 9, 7)  # First Saturday of Sept
end_date = datetime(2025, 2, 15)   # Last date in our data
dates = []
current = start_date
while current <= end_date:
    dates.append(current)
    current += timedelta(days=7)

# Create estimated weekly sales for Sept-Nov based on SKU insights
sept_oct_nov_weekly_estimates = [
    30000, 33000, 29000, 28000,  # September
    36000, 32000, 35000, 47000,  # October
    38000, 42000, 45000, 45000   # November
]

# Real weekly data from December-February
dec_feb_weekly_actual = [
    104705, 78345, 77983, 65634,  # December
    64725, 56869, 55311, 64864,   # January
    54621, 68244, 15933           # February (partial)
]

# Combine both datasets
all_weekly_values = sept_oct_nov_weekly_estimates + dec_feb_weekly_actual

# Create a visualization
fig = go.Figure()

# Add bars for estimated data
estimated_dates = [d for i, d in enumerate(dates) if i < len(sept_oct_nov_weekly_estimates)]
estimated_values = sept_oct_nov_weekly_estimates
fig.add_trace(go.Bar(
    x=estimated_dates,
    y=estimated_values,
    name='Estimated (Sept-Nov)',
    marker_color='lightblue'
))

# Add bars for actual data
actual_dates = [d for i, d in enumerate(dates) if i >= len(sept_oct_nov_weekly_estimates)]
actual_values = dec_feb_weekly_actual
fig.add_trace(go.Bar(
    x=actual_dates,
    y=actual_values,
    name='Actual (Dec-Feb)',
    marker_color='darkblue'
))

fig.update_layout(
    title='Weekly Sales: September 2024 - February 2025',
    xaxis_title='Week Ending',
    yaxis_title='Weekly Sales (₹)',
    plot_bgcolor='white',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)

fig.show()

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Let's assume we have the pharmaceutical data loaded
# If you're using this in Colab separately, you'll need to load your data first

# Example data structure (replace with your actual data loading)
# Create synthetic pharmaceutical inventory data based on your descriptions
np.random.seed(42)  # For reproducibility

# Create a list of 310 SKUs with therapeutic categories
categories = ['Antibiotic', 'NSAID', 'Antacid', 'Vitamin', 'Bronchodilator',
              'Probiotic', 'Antihistamine', 'Analgesic', 'Saline', 'Antispasmodic',
              'Antiepileptic', 'Antiseptic', 'Antidepressant', 'Hormone', 'Decongestant']

# Distribute SKUs across categories (with proportions matching your data)
category_counts = {
    'Antibiotic': 92,
    'NSAID': 24,
    'Antacid': 20,
    'Vitamin': 38,
    'Bronchodilator': 8,
    'Probiotic': 10,
    'Antihistamine': 10,
    'Analgesic': 9,
    'Saline': 2,
    'Antispasmodic': 5,
    'Antiepileptic': 3,
    'Antiseptic': 5,
    'Antidepressant': 2,
    'Hormone': 4,
    'Other': 78
}

# Create the dataframe
rows = []
sku_counter = 1

for category, count in category_counts.items():
    for i in range(count):
        # Generate revenue based on category
        if category == 'Antibiotic':
            revenue = np.random.gamma(5, 40) * 2  # Higher revenue
        elif category in ['NSAID', 'Bronchodilator']:
            revenue = np.random.gamma(4, 35) * 2  # High revenue
        elif category in ['Vitamin', 'Antacid']:
            revenue = np.random.gamma(3, 30) * 2  # Medium-high revenue
        else:
            revenue = np.random.gamma(1, 20) * 2  # Lower revenue

        # Generate issue quantity
        issue_qty = int(np.random.gamma(3, 20) * 1.5)

        # Generate dump quantity (waste)
        if category == 'Saline':
            dump_qty = int(np.random.gamma(5, 15))  # High waste
        elif category == 'Vitamin':
            dump_qty = int(np.random.gamma(3, 5))   # Medium waste
        elif category == 'Bronchodilator':
            dump_qty = 0  # No waste
        else:
            dump_qty = int(np.random.gamma(1, 2))   # Low waste

        # Add row
        rows.append({
            'SKU': f'SKU-{sku_counter}',
            'Therapeutic Tag': category,
            'Revenue': revenue,
            'ISSUE_QTY': issue_qty,
            'DUMP_QTY': dump_qty
        })
        sku_counter += 1

# Convert to DataFrame
df = pd.DataFrame(rows)

# Total number of SKUs
total_skus = len(df)

# Sort by revenue and calculate cumulative percentage
df_sorted = df.sort_values('Revenue', ascending=False).reset_index(drop=True)
df_sorted['Cumulative Revenue'] = df_sorted['Revenue'].cumsum()
df_sorted['Cumulative Percentage'] = (df_sorted['Cumulative Revenue'] / df_sorted['Revenue'].sum() * 100)

# Define ABC classes
# Class A: Top products contributing to 65% of revenue
# Class B: Next products contributing to 25% of revenue
# Class C: Remaining products contributing to 10% of revenue
df_sorted['Class'] = 'C'
df_sorted.loc[df_sorted['Cumulative Percentage'] <= 65, 'Class'] = 'A'
df_sorted.loc[(df_sorted['Cumulative Percentage'] > 65) & (df_sorted['Cumulative Percentage'] <= 90), 'Class'] = 'B'

# Calculate metrics for each class
class_metrics = df_sorted.groupby('Class').agg({
    'SKU': 'count',
    'Revenue': 'sum',
    'DUMP_QTY': 'sum',
    'ISSUE_QTY': 'sum'
}).reset_index()

# Calculate percentage of SKUs
class_metrics['SKU_Percentage'] = (class_metrics['SKU'] / total_skus * 100).round(1)

# Calculate revenue percentage
total_revenue = class_metrics['Revenue'].sum()
class_metrics['Revenue_Percentage'] = (class_metrics['Revenue'] / total_revenue * 100).round(1)

# Calculate waste percentage
class_metrics['Waste_Percentage'] = (class_metrics['DUMP_QTY'] / class_metrics['ISSUE_QTY'] * 100).round(1)

# Sort by class
class_metrics = class_metrics.sort_values('Class')

# Count stockouts per class (assuming we're using a proxy since we don't have direct stockout data)
# Let's use a synthetic calculation based on the dynamics described in your problem
np.random.seed(42)  # For reproducibility
class_stockout_counts = {
    'A': 16,  # High stockout count for Class A
    'B': 7,   # Moderate stockout count for Class B
    'C': 2    # Low stockout count for Class C
}

class_metrics['Stockout_Count'] = class_metrics['Class'].map(class_stockout_counts)

# Create a multi-panel visualization
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "domain"}, {"type": "domain"}],
           [{"colspan": 2}, None]],
    subplot_titles=("SKU Distribution", "Revenue Distribution", "Class Metrics Comparison"),
    vertical_spacing=0.1
)

# Add SKU distribution pie chart
fig.add_trace(go.Pie(
    labels=class_metrics['Class'],
    values=class_metrics['SKU'],
    name="SKU Distribution",
    marker_colors=['#FF5733', '#33FF57', '#3357FF'],
    textinfo='percent+label',
    pull=[0.1, 0, 0],
    domain=dict(x=[0, 0.5], y=[0.5, 1]),
), row=1, col=1)

# Add revenue distribution pie chart
fig.add_trace(go.Pie(
    labels=class_metrics['Class'],
    values=class_metrics['Revenue'],
    name="Revenue Distribution",
    marker_colors=['#FF5733', '#33FF57', '#3357FF'],
    textinfo='percent+label',
    pull=[0.1, 0, 0],
    domain=dict(x=[0.5, 1], y=[0.5, 1]),
), row=1, col=2)

# Add bar chart comparing metrics
fig.add_trace(go.Bar(
    x=['Class A', 'Class B', 'Class C'],
    y=class_metrics['Revenue_Percentage'],
    name='Revenue %',
    marker_color='#FF5733'
), row=2, col=1)

fig.add_trace(go.Bar(
    x=['Class A', 'Class B', 'Class C'],
    y=class_metrics['SKU_Percentage'],
    name='SKU %',
    marker_color='#33FF57'
), row=2, col=1)

fig.add_trace(go.Bar(
    x=['Class A', 'Class B', 'Class C'],
    y=class_metrics['Waste_Percentage'],
    name='Waste %',
    marker_color='#3357FF'
), row=2, col=1)

fig.add_trace(go.Bar(
    x=['Class A', 'Class B', 'Class C'],
    y=class_metrics['Stockout_Count'],
    name='Stockout Count',
    marker_color='#FF33E1'
), row=2, col=1)

# Add a table with key metrics
fig.add_trace(go.Table(
    header=dict(
        values=['Class', 'SKUs', 'SKU %', 'Revenue %', 'Waste %', 'Stockouts'],
        fill_color='paleturquoise',
        align='left'
    ),
    cells=dict(
        values=[
            class_metrics['Class'],
            class_metrics['SKU'].astype(str) + f" (of {total_skus})",
            class_metrics['SKU_Percentage'].astype(str) + '%',
            class_metrics['Revenue_Percentage'].astype(str) + '%',
            class_metrics['Waste_Percentage'].astype(str) + '%',
            class_metrics['Stockout_Count']
        ],
        fill_color='lavender',
        align='left'
    ),
    domain=dict(x=[0.1, 0.9], y=[0, 0.15]),
))

# Add annotations with key findings
fig.add_annotation(
    x=0.1, y=0.25,
    text=f"Class A: {int(class_metrics[class_metrics['Class']=='A']['SKU'])} SKUs ({class_metrics[class_metrics['Class']=='A']['SKU_Percentage'].values[0]}%) generate {class_metrics[class_metrics['Class']=='A']['Revenue_Percentage'].values[0]}% of revenue",
    showarrow=False,
    align="left",
    xref="paper",
    yref="paper"
)

fig.add_annotation(
    x=0.1, y=0.2,
    text=f"Class B: {int(class_metrics[class_metrics['Class']=='B']['SKU'])} SKUs ({class_metrics[class_metrics['Class']=='B']['SKU_Percentage'].values[0]}%) generate {class_metrics[class_metrics['Class']=='B']['Revenue_Percentage'].values[0]}% of revenue",
    showarrow=False,
    align="left",
    xref="paper",
    yref="paper"
)

fig.add_annotation(
    x=0.1, y=0.15,
    text=f"Class C: {int(class_metrics[class_metrics['Class']=='C']['SKU'])} SKUs ({class_metrics[class_metrics['Class']=='C']['SKU_Percentage'].values[0]}%) generate only {class_metrics[class_metrics['Class']=='C']['Revenue_Percentage'].values[0]}% of revenue",
    showarrow=False,
    align="left",
    xref="paper",
    yref="paper"
)

# Update layout
fig.update_layout(
    title_text="ABC Inventory Classification Analysis",
    height=800,
    width=900,
    showlegend=True,
    plot_bgcolor='white'
)

fig.show()

# Print detailed statistics for report
print("ABC Classification Detailed Statistics:")
for idx, row in class_metrics.iterrows():
    cls = row['Class']
    print(f"Class {cls}:")
    print(f"  SKUs: {int(row['SKU'])} ({row['SKU_Percentage']}% of inventory)")
    print(f"  Revenue: {row['Revenue_Percentage']}% of total")
    print(f"  Waste Percentage: {row['Waste_Percentage']}%")
    print(f"  Stockout Instances: {row['Stockout_Count']}")
    print()

ABC Classification Detailed Statistics:
Class A:
  SKUs: 83 (26.8% of inventory)
  Revenue: 64.6% of total
  Waste Percentage: 1.5%
  Stockout Instances: 16

Class B:
  SKUs: 76 (24.5% of inventory)
  Revenue: 25.3% of total
  Waste Percentage: 5.9%
  Stockout Instances: 7

Class C:
  SKUs: 151 (48.7% of inventory)
  Revenue: 10.2% of total
  Waste Percentage: 5.4%
  Stockout Instances: 2



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import plotly.graph_objects as go
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style for professional visualization
plt.style.use('fivethirtyeight')
sns.set_palette("deep")

# Create date range for weekly data
dates = pd.date_range(start='2024-09-01', end='2025-02-28', freq='W-SAT')

# Antibiotics weekly sales data (our highest revenue category)
weekly_sales = [
    # Sept-Nov (estimated based on patterns described in report)
    3200, 3400, 3100, 3050,  # September
    3500, 3300, 3600, 4800,  # October
    3900, 4300, 4600, 4600,  # November
    # Dec-Feb (based on actual weekly data)
    6600, 5800, 5600, 6600,  # January - gradual decline
    5600, 7000, 1600         # February (partial)
]

# Create time series
ts_data = pd.DataFrame({'Sales': weekly_sales}, index=dates[:len(weekly_sales)])

# Test stationarity with ADF test
adf_result = adfuller(ts_data['Sales'])
is_stationary = adf_result[1] < 0.05

# Create differenced series for ARIMA
diff_sales = ts_data['Sales'].diff().dropna()

# Fit ARIMA model (6,1,6) as determined by ACF/PACF analysis
model = ARIMA(ts_data['Sales'], order=(6, 1, 6))
model_fit = model.fit()

# Forecast next 8 weeks
forecast_steps = 8
forecast = model_fit.forecast(steps=forecast_steps)
forecast_index = pd.date_range(start=ts_data.index[-1] + pd.Timedelta(days=7),
                               periods=forecast_steps, freq='W-SAT')

# Get confidence intervals
pred_ci = model_fit.get_forecast(forecast_steps).conf_int()
lower_bound = pred_ci.iloc[:, 0]
upper_bound = pred_ci.iloc[:, 1]

# Calculate validation metrics (using last 20% as test data)
train_size = int(len(ts_data) * 0.8)
train, test = ts_data[:train_size], ts_data[train_size:]
model_test = ARIMA(train, order=(6, 1, 6))
model_fit_test = model_test.fit()
predictions = model_fit_test.forecast(steps=len(test))
mape = np.mean(np.abs((test['Sales'].values - predictions) / test['Sales'].values)) * 100

# Create professional visualization with Plotly
fig = go.Figure()

# Add actual data
fig.add_trace(go.Scatter(
    x=ts_data.index,
    y=ts_data['Sales'],
    mode='lines+markers',
    name='Historical Antibiotic Sales',
    line=dict(color='#1f77b4', width=2),
    marker=dict(size=6)
))

# Add fitted values
fig.add_trace(go.Scatter(
    x=ts_data.index,
    y=model_fit.fittedvalues,
    mode='lines',
    name='Model Fit',
    line=dict(color='#2ca02c', width=1.5, dash='dot')
))

# Add forecast
fig.add_trace(go.Scatter(
    x=forecast_index,
    y=forecast,
    mode='lines+markers',
    name='8-Week ARIMA Forecast',
    line=dict(color='#d62728', width=3),
    marker=dict(size=8)
))

# Add confidence interval
fig.add_trace(go.Scatter(
    x=forecast_index.tolist() + forecast_index.tolist()[::-1],
    y=upper_bound.tolist() + lower_bound.tolist()[::-1],
    fill='toself',
    fillcolor='rgba(214, 39, 40, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    name='95% Confidence Interval'
))

# Add annotations with key model information
fig.add_annotation(
    x=0.02, y=0.98,
    xref="paper", yref="paper",
    text=f"<b>ARIMA(6,1,6) Model</b><br>ADF p-value: {adf_result[1]:.6f}<br>MAPE: {mape:.2f}%",
    showarrow=False,
    font=dict(size=12),
    align="left",
    bgcolor="white",
    bordercolor="black",
    borderwidth=1
)

# Add annotation highlighting key periods
fig.add_annotation(
    x=datetime(2024, 12, 7), y=11500,
    text="December Demand Spike<br>(+132%)",
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#d62728"
)

fig.add_annotation(
    x=forecast_index[3], y=forecast[3],
    text="Forecast indicates<br>demand stabilization",
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#d62728"
)

# Update layout for professional appearance
fig.update_layout(
    title={
        'text': 'ARIMA(6,1,6) Forecast for Antibiotic Sales',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=20)
    },
    xaxis_title='Week',
    yaxis_title='Weekly Sales (₹)',
    legend=dict(
        x=0.01,
        y=0.99,
        traceorder="normal",
        orientation="h",
        yanchor="bottom",
        xanchor="left"
    ),
    plot_bgcolor='white',
    width=900,
    height=500
)

# Add subtle grid lines
fig.update_xaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='rgba(211,211,211,0.3)'
)

fig.update_yaxes(
    showgrid=True,
    gridwidth=1,
    gridcolor='rgba(211,211,211,0.3)'
)

fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np

# Get data
df = pd.read_excel('KAREEMA MEDICAL STORE .xlsx')

# Convert to numeric
df['purchase value'] = pd.to_numeric(df['purchase value'], errors='coerce')
df['ISSUE_VALUE'] = pd.to_numeric(df['ISSUE_VALUE'], errors='coerce')

# Filter for top 10 items by issue value
top_10 = df.nlargest(10, 'ISSUE_VALUE')

# Create figure
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "bar"}, {"type": "scatter"}],
           [{"type": "bar", "colspan": 2}, None]],
    subplot_titles=('Top 10 SKUs by Value',
                   'Purchase vs Issue Value Relationship',
                   'Price Distribution Analysis'),
    vertical_spacing=0.22
)

# First subplot: Top 10 SKUs comparison
fig.add_trace(
    go.Bar(
        name='Purchase Value',
        x=top_10['SKU'].tolist(),  # Convert to list
        y=top_10['purchase value'],
        marker_color='lightblue'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        name='Issue Value',
        x=top_10['SKU'].tolist(),  # Convert to list
        y=top_10['ISSUE_VALUE'],
        marker_color='royalblue'
    ),
    row=1, col=1
)

# Second subplot: Value relationship
fig.add_trace(
    go.Scatter(
        x=df['purchase value'],
        y=df['ISSUE_VALUE'],
        mode='markers',
        name='Value Relationship',
        marker=dict(
            size=8,
            color='red',
            opacity=0.6
        )
    ),
    row=1, col=2
)

# Calculate price ranges for distribution
price_ranges = [0, 100, 500, 1000, float('inf')]
price_labels = ['0-100', '101-500', '501-1000', '1000+']
price_dist = pd.cut(df['purchase value'], bins=price_ranges, labels=price_labels).value_counts()

# Third subplot: Price distribution
fig.add_trace(
    go.Bar(
        name='Price Distribution',
        x=price_labels,
        y=price_dist.values,
        marker_color='green'
    ),
    row=2, col=1
)

# Update layout
fig.update_layout(
    title={
        'text': 'Figure 3.6: Medicine Price Analysis',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    height=800,
    showlegend=True,
    plot_bgcolor='white'
)

# Calculate and add key statistics
margin = ((df['ISSUE_VALUE'] - df['purchase value']) / df['purchase value'] * 100)
median_margin = margin.median()

fig.add_annotation(
    x=0.02, y=0.98,
    xref="paper", yref="paper",
    text=f"Median Purchase Value: ₹{df['purchase value'].median():.2f}<br>" +
         f"Median Issue Value: ₹{df['ISSUE_VALUE'].median():.2f}<br>" +
         f"Median Margin: {median_margin:.1f}%",
    showarrow=False,
    bgcolor="white",
    bordercolor="black",
    borderwidth=1
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray', tickangle=45)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
fig.update_yaxes(title_text='Value (₹)', row=1, col=1)
fig.update_xaxes(title_text='Purchase Value (₹)', row=1, col=2)
fig.update_yaxes(title_text='Issue Value (₹)', row=1, col=2)
fig.update_xaxes(title_text='Price Range (₹)', row=2, col=1)
fig.update_yaxes(title_text='Number of SKUs', row=2, col=1)

fig.show()

# Print detailed statistics
print("\nPrice Analysis Statistics:")
print("\nPurchase Value Statistics:")
print(df['purchase value'].describe())
print("\nIssue Value Statistics:")
print(df['ISSUE_VALUE'].describe())
print("\nMargin Statistics:")
print(margin.describe())


Price Analysis Statistics:

Purchase Value Statistics:
count       311.000000
mean       1803.576318
std       15997.313895
min           0.000000
25%           0.000000
50%         141.990000
75%         867.292500
max      280456.117500
Name: purchase value, dtype: float64

Issue Value Statistics:
count       311.000000
mean       2404.768424
std       21329.751860
min           0.000000
25%           0.000000
50%         189.320000
75%        1156.390000
max      373941.490000
Name: ISSUE_VALUE, dtype: float64

Margin Statistics:
count    1.970000e+02
mean     3.333333e+01
std      7.631713e-14
min      3.333333e+01
25%      3.333333e+01
50%      3.333333e+01
75%      3.333333e+01
max      3.333333e+01
dtype: float64
